In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import glob
import os
import re
import warnings
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
from scipy import stats
import sklearn
import sklearn.preprocessing
warnings.filterwarnings("ignore")

Orgenizing the data for training

In [2]:
names = glob.glob('datasets\Training\*')
training_data  = pd.DataFrame(data=pd.read_csv(glob.glob(str(names[0])+"\*")[-1]))
training_data = training_data.loc[training_data['Time']-training_data['Time'][0]>7]
training_data = pd.concat([training_data,pd.read_csv('HandRight.csv')],sort=False)
training_data['Type'] = str(re.search(r'[a-zA-Z].*[0-9](.*)',glob.glob(str(names[0])+"\*")[-1]).group(1)[:-4])

csv_t = pd.read_csv(glob.glob(str(names[0])+"\*")[-2])
csv_t = csv_t.loc[csv_t['Time']-csv_t['Time'][0]>7]
csv_t['Type'] =  str(re.search(r'[a-zA-Z].*[0-9](.*)',glob.glob(str(names[0])+"\*")[-2]).group(1)[:-4])
training_data = pd.concat([training_data, csv_t],sort=False)

csv_q = pd.read_csv(glob.glob(str(names[0])+"\*")[-3])
csv_q = csv_q.loc[csv_q['Time']-csv_q['Time'][0]>7]
csv_q['Type'] =  str(re.search(r'[a-zA-Z].*[0-9](.*)',glob.glob(str(names[0])+"\*")[-3]).group(1)[:-4])
training_data = pd.concat([training_data, csv_q],sort=False)
print(training_data.shape)
for i in list(glob.glob('datasets\Training\*')):
    names1 = glob.glob(str(i)+"\*") # List of names of all participants in the experiment.
    
    csv1 = pd.read_csv(names1[-1])
    csv1 = csv1.loc[csv1['Time']-csv1['Time'][0]>7]
    csv1['Type'] = str(re.search(r'[a-zA-Z].*[0-9](.*)',names1[-1]).group(1)[:-4])
    if 'Alone' in names1[-1]:
        training_data = pd.concat([training_data, csv1, pd.read_csv('HandRight.csv')],sort=False)
    else:
        training_data = pd.concat([training_data, csv1],sort=False)
        
    csv2 = pd.read_csv(names1[-2])
    csv2 = csv2.loc[csv2['Time']-csv2['Time'][0]>7]
    csv2['Type'] = str(re.search(r'[a-zA-Z].*[0-9](.*)',names1[-2]).group(1)[:-4])
    if 'Alone' in names1[-2]:
        training_data = pd.concat([training_data, csv2, pd.read_csv('HandRight.csv')],sort=False)
    else: 
        training_data = pd.concat([training_data, csv2],sort=False)
    
    csv3 = pd.read_csv(names1[-3])
    csv3 = csv3.loc[csv3['Time']-csv3['Time'][0]>7]
    csv3['Type'] = str(re.search(r'[a-zA-Z].*[0-9](.*)',names1[-3]).group(1)[:-4])
    if 'Alone' in names1[-3]:
        training_data = pd.concat([training_data, csv3, pd.read_csv('HandRight.csv')],sort=False)
    else:
        training_data = pd.concat([training_data, csv3],sort=False)

training_data['Type'].replace({'Alone':1,'Spontan':2,'Sync':3},inplace=True)

del training_data[" # hands"]
del training_data[" Hand Type"]
print(training_data.shape)
training_data = training_data.dropna(how = 'any')

(25329, 23)
(241553, 21)


Orgenizing the data for testing

In [3]:
names = glob.glob('datasets\Validation\*')
testing_data  = pd.DataFrame(data=pd.read_csv(glob.glob(str(names[0])+"\*")[-1]))
testing_data = pd.concat([testing_data,pd.read_csv('HandRight.csv')],sort=False)
testing_data['Type'] = str(re.search(r'[a-zA-Z].*[0-9](.*)',glob.glob(str(names[0])+"\*")[-1]).group(1)[:-4])

csv_t = pd.read_csv(glob.glob(str(names[0])+"\*")[-2])
csv_t['Type'] =  str(re.search(r'[a-zA-Z].*[0-9](.*)',glob.glob(str(names[0])+"\*")[-2]).group(1)[:-4])
testing_data = pd.concat([testing_data, csv_t],sort=False)

csv_q = pd.read_csv(glob.glob(str(names[0])+"\*")[-3])
csv_q['Type'] =  str(re.search(r'[a-zA-Z].*[0-9](.*)',glob.glob(str(names[0])+"\*")[-3]).group(1)[:-4])
testing_data = pd.concat([testing_data, csv_q],sort=False)
print(testing_data.shape)
for i in list(glob.glob('datasets\Validation\*')):
    names1 = glob.glob(str(i)+"\*") # List of names of all participants in the experiment.
    
    csv1 = pd.read_csv(names1[-1])
    csv1 = csv1.loc[csv1['Time']-csv1['Time'][0]>7]
    csv1['Type'] = str(re.search(r'[a-zA-Z].*[0-9](.*)',names1[-1]).group(1)[:-4])
    if 'Alone' in names1[-1]:
        testing_data = pd.concat([testing_data, csv1,pd.read_csv('HandRight.csv')],sort=False)
    else:
        testing_data = pd.concat([testing_data, csv1],sort=False)
        
    csv2 = pd.read_csv(names1[-2])
    csv2 = csv2.loc[csv2['Time']-csv2['Time'][0]>7]
    csv2['Type'] = str(re.search(r'[a-zA-Z].*[0-9](.*)',names1[-2]).group(1)[:-4])
    if 'Alone' in names1[-2]:
        testing_data = pd.concat([testing_data, csv2,pd.read_csv('HandRight.csv')],sort=False)
    else:
        testing_data = pd.concat([testing_data, csv2],sort=False)
    csv3 = pd.read_csv(names1[-3])
    csv3 = csv3.loc[csv3['Time']-csv3['Time'][0]>7]
    csv3['Type'] = str(re.search(r'[a-zA-Z].*[0-9](.*)',names1[-3]).group(1)[:-4])
    if 'Alone' in names1[-3]:
        testing_data = pd.concat([testing_data,csv3,pd.read_csv('HandRight.csv')],sort=False)
    else:
        testing_data = pd.concat([testing_data, csv3],sort=False)
    
testing_data['Type'].replace({'Alone':1,'Spontan':2,'Sync':3},inplace=True)
del testing_data[" # hands"]
del testing_data[" Hand Type"]
print(testing_data.shape)
testing_data = testing_data.dropna(how ='any')

(27327, 23)
(92751, 21)


In [4]:
X_train = training_data.loc[:, training_data.columns != 'Type']
y_train = training_data[["Type"]]
X_test = testing_data.loc[:, testing_data.columns != 'Type']
y_test = testing_data[["Type"]]
X = pd.concat([X_train,X_test],sort=False)

In [5]:
pca = PCA(n_components=5)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
fit = pca.fit(X)
# summarize components
print(fit.explained_variance_ratio_)

[9.99792186e-01 1.03296605e-04 3.20631464e-05 3.12073114e-05
 1.31946046e-05]


In [6]:
adaboost = AdaBoostClassifier(n_estimators=100, base_estimator= None,learning_rate=1, random_state = 1)
adaboost.fit(X_train,y_train)
y_pred = adaboost.predict(X_test)
cm = confusion_matrix(y_test,y_pred)
accuracy = float(cm.diagonal().sum())/len(y_test)
print("\nAccuracy Of AdaBoost For The Given Dataset : ", accuracy)
print("\nF-score wighted : ", f1_score(y_test,y_pred,average='weighted'))


Accuracy Of AdaBoost For The Given Dataset :  0.39736433511696245

F-score wighted :  0.30419203586044735


In [7]:
my_scaler = sklearn.preprocessing.StandardScaler()
my_scaler.fit(X_train)
X_train = my_scaler.transform(X_train)
X_test = my_scaler.transform(X_test)
classifier = sklearn.neighbors.KNeighborsClassifier(n_neighbors=10)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
score = sklearn.metrics.accuracy_score(y_test, y_pred)
print('Accuracy :', score)

Accuracy : 0.44740860504805535
